# ETL Project

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Pulled population density of states of the United States from Wikipedia using read_html
url = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density'

In [3]:
# Read_html provided multiple tables. Table needed was at index 0
# The table returned had a tuple for table headers. Renamed columns for easier management of data frame
pop_density = pd.read_html(url)
pop_density[0].columns = ['state',
'Den_Rank(all)',
'Den_Rank(50 states)',
'den_per_sqm',
'Den_perkm2',
'Pop_Rank',
'pop_numbers',
'Land_Rank',
'land_sqm',
'Land_km2']
pop_density[0].head()

,state,Den_Rank(all),Den_Rank(50 states),den_per_sqm,Den_perkm2,Pop_Rank,pop_numbers,Land_Rank,land_sqm,Land_km2
0,District of Columbia,1,—,11011,4251,50,672228,56,61,158.0
1,New Jersey,2,1,1218,470,11,8958013,46,7354,19046.8
2,Puerto Rico,3,—,1046,404,29,3680058,49,3515,9103.8
3,Rhode Island,4,2,1021,394,44,1056298,51,1034,2678.0
4,Massachusetts,5,3,871,336,15,6794422,45,7800,20201.9


In [4]:
# Determined which columns were needed
pop_density_df = pop_density[0][['state', 'den_per_sqm', 'pop_numbers', 'land_sqm']].copy()
# One row had a footnote tag which caused erroneous data e.g. 155,959[5] which needed to be cleaned
pop_density_df['land_sqm']= pop_density_df['land_sqm'].str.replace(r"\[.*\]","").str.replace(r",","")
pop_density_df.set_index('state', inplace=True)
pop_density_df.head()

,den_per_sqm,pop_numbers,land_sqm
state,,,
District of Columbia,11011,672228,61
New Jersey,1218,8958013,7354
Puerto Rico,1046,3680058,3515
Rhode Island,1021,1056298,1034
Massachusetts,871,6794422,7800


In [5]:
# Pulled GDP of states of the United States from Wikipedia and captured as a CSV file
gdp_per_capita = "./Resources/gdp_per_capita.csv"
gdp_df = pd.read_csv(gdp_per_capita)
gdp_df.head()

,Rank,State,2018,2017,2016,2015,2014,2013,2012,2011
0,NaN,District of Columbia,160472,159227,159395,159497,163274,166870,168030,166178
1,1.0,Massachusetts,65545,64507,62510,61882,62456,61769,60808,59178
2,2.0,New York,64579,64093,63420,62444,62841,61185,61267,59481
3,3.0,Connecticut,64511,63747,62236,62550,63502,63638,64906,65574
4,4.0,Alaska,63971,67705,67411,69700,73478,70573,69564,72204


In [6]:
# Determined which columns were needed
new_gdp_df = gdp_df[['State', '2015']].copy()
# Targeted year 2015 to match data from the population density data and renamed to 'gdp'
new_gdp_df = new_gdp_df.rename(columns={"2015": "gdp", "State": "state"})
new_gdp_df.set_index('state', inplace=True)
new_gdp_df.head()

,gdp
state,
District of Columbia,159497
Massachusetts,61882
New York,62444
Connecticut,62550
Alaska,69700


In [7]:
# Connect to postgres DB
connection_string = "postgres:postgres@localhost:5432/states_db"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

['states_gdp', 'states_pop']

In [9]:
# Load data into states_db.states_pop
engine.execute('DELETE FROM states_pop', con=engine)
pop_density_df.to_sql(name='states_pop', con=engine, if_exists='append', index=True)

In [10]:
# Load data into states_db.states_gdp
engine.execute('DELETE FROM states_gdp', con=engine)
new_gdp_df.to_sql(name='states_gdp', con=engine, if_exists='append', index=True)

In [11]:
# Query DB with an inner join to bring data together
states_all_df = pd.read_sql_query('SELECT states_gdp.state, states_gdp.gdp, states_pop.den_per_sqm, states_pop.pop_numbers, states_pop.land_sqm  FROM states_pop INNER JOIN states_gdp ON states_gdp.state = states_pop.state', con=engine)
states_all_df.head(100)

,state,gdp,den_per_sqm,pop_numbers,land_sqm
0,District of Columbia,159497,11011,672228,61
1,New Jersey,55750,1218,8958013,7354
2,Rhode Island,46356,1021,1056298,1034
3,Massachusetts,61882,871,6794422,7800
4,Connecticut,62550,741,3590886,4842
5,Maryland,53765,618,6006401,9707
6,Delaware,60719,485,945934,1949
7,New York,62444,420,19795791,47126
8,Florida,38023,378,20271272,53625
9,Pennsylvania,48278,286,12802503,44743
